In [1]:
from pyspark import SparkContext
from datetime import datetime
from dateutil.parser import parse
from pyspark.sql.functions import udf, to_date, to_utc_timestamp
import numpy as np

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1579133582147_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *


from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier


ss = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
station_path = 's3://msds694.proj/data/station.csv'
status_path = 's3://msds694.proj/data/status.csv'
trip_path = 's3://msds694.proj/data/trip.csv'
weather_path = 's3://msds694.proj/data/weather.csv'
population_path = 's3://msds694.proj/data/station_population.csv'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
station_population=ss.read.csv(population_path,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
station_population.cache().show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+---------+----------+----------+--------+-----------------+----+----+
| id|                name|      lat|      long|dock_count|    city|installation_date| _c7| _c8|
+---+--------------------+---------+----------+----------+--------+-----------------+----+----+
|  2|San Jose Diridon ...|37.329732|-121.90178|        27|San Jose|           8/6/13|null|null|
|  3|San Jose Civic Ce...|37.330698|-121.88898|        15|San Jose|           8/5/13|null|null|
|  4|Santa Clara at Al...|37.333988| -121.8949|        11|San Jose|           8/6/13|null|null|
|  5|    Adobe on Almaden|37.331415| -121.8932|        19|San Jose|           8/5/13|null|null|
|  6|    San Pedro Square|37.336721|-121.89407|        15|San Jose|           8/7/13|null|null|
+---+--------------------+---------+----------+----------+--------+-----------------+----+----+
only showing top 5 rows

In [8]:
status=ss.read.csv(status_path,header=True)
status.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+---------------+-------------------+
|station_id|bikes_available|docks_available|               time|
+----------+---------------+---------------+-------------------+
|         2|              2|             25|2013/08/29 12:06:01|
|         2|              2|             25|2013/08/29 12:07:01|
|         2|              2|             25|2013/08/29 12:08:01|
|         2|              2|             25|2013/08/29 12:09:01|
|         2|              2|             25|2013/08/29 12:10:01|
+----------+---------------+---------------+-------------------+
only showing top 5 rows

In [9]:
status_rdd = sc.textFile(status_path)
weather_rdd = sc.textFile(weather_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
STATION_ID = 0
BIKES_AVAILABLE = 1
DOCKS_AVAILABLE = 2
TIME = 3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def change_col(x,col_num,func,**kwargs):
    x[col_num] = func(x[col_num],**kwargs)
    return x

# def makekey(x,*args):
#     key = tuple([item for idx,item in enumerate(x) if idx in args])
#     return (key,x)

# def flatten(x,iterations=1):
    
#     return sum(x[1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
status_rdd.cache().take(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['station_id,bikes_available,docks_available,time', '2,2,25,2013/08/29 12:06:01']

In [13]:
status_rdd_processed = (status_rdd.filter(lambda x: not x.startswith('station_id'))
                                  .map(lambda x:x.split(','))
                                  .map(lambda x:x[:TIME]+[x[TIME].replace('/','-')])
                                  .map(lambda x: change_col(x,TIME,parse))
                                  .map(lambda x:x+[datetime(x[TIME].year,x[TIME].month,x[TIME].day,x[TIME].hour)]
                                       +[x[TIME].hour]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
def toDoubleSafe(v):
    try:
        return float(v)
    except ValueError:
        return float(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
weather_rdd = sc.textFile(weather_path, 4)\
            .filter(lambda x: not x.startswith('date'))\
                                  .map(lambda x:x.split(','))\
                                  .map(lambda x:[x[0],toDoubleSafe(x[2]),toDoubleSafe(x[19])])\
                                  .map(lambda x: change_col(x,0,parse))#.map(lambda x:[x[0]]+[float(x[2])])
weather_rdd.take(3)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[datetime.datetime(2013, 8, 29, 0, 0), 68.0, 0.0], [datetime.datetime(2013, 8, 30, 0, 0), 69.0, 0.0], [datetime.datetime(2013, 8, 31, 0, 0), 64.0, 0.0]]

In [72]:
weather_schema = StructType([
    StructField("date",DateType(),False),
    StructField("mean_temp",FloatType(),False),
    StructField("rain_identifier",FloatType(),False)])
    
weather_df = ss.createDataFrame(weather_rdd, schema=weather_schema)
weather_df.show(3)
# weather_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+---------------+
|      date|mean_temp|rain_identifier|
+----------+---------+---------------+
|2013-08-29|     68.0|            0.0|
|2013-08-30|     69.0|            0.0|
|2013-08-31|     64.0|            0.0|
+----------+---------+---------------+
only showing top 3 rows

In [136]:
weather_df_processed = weather_df.select(weather_df.date.cast(StringType()).alias('date'),
                                        weather_df.mean_temp.cast(DoubleType()).alias('temp_float'),
                                        weather_df.rain_identifier.cast(DoubleType()).alias('rain_level_float'))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [137]:
weather_df_processed = weather_df_processed.groupBy('date').agg(mean('temp_float'),mean('rain_level_float'))
weather_df_processed = weather_df_processed.withColumnRenamed('avg(temp_float)','temp_float').withColumnRenamed('avg(rain_level_float)','rain_level_float')
weather_df_processed.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+--------------------+
|      date|temp_float|    rain_level_float|
+----------+----------+--------------------+
|2014-02-16|      56.0|0.021999999508261682|
|2014-02-22|      56.4|                 0.0|
|2014-12-13|      50.4|0.003999999910593033|
|2015-05-01|      54.2|                 0.0|
|2014-05-27|      63.0|                 0.0|
+----------+----------+--------------------+
only showing top 5 rows

In [138]:
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col

weather_df_stats = weather_df_processed.select(
    _mean(col('rain_level_float')).alias('mean')
).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [139]:
mean_rain = weather_df_stats[0][0]
mean_rain

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.020120054476298548

In [140]:
from pyspark.sql.functions import *
check_rain = udf(lambda x: x > mean_rain)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [141]:
weather_df_processed = weather_df_processed.select('date', 'temp_float', check_rain('rain_level_float') \
                                            .alias('rain_identifier'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [142]:
weather_df_processed.summary()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, date: string, temp_float: string, rain_identifier: string]

In [40]:
status_rdd_processed.cache().take(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['2', '2', '25', datetime.datetime(2013, 8, 29, 12, 6, 1), datetime.datetime(2013, 8, 29, 12, 0), 12], ['2', '2', '25', datetime.datetime(2013, 8, 29, 12, 7, 1), datetime.datetime(2013, 8, 29, 12, 0), 12], ['2', '2', '25', datetime.datetime(2013, 8, 29, 12, 8, 1), datetime.datetime(2013, 8, 29, 12, 0), 12]]

In [41]:
def is_weekend(date):
    weekday_name = date.strftime('%A')
    if weekday_name in ['Saturday','Sunday']:
        return True
    else:
        return False

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
status_rdd_week = status_rdd_processed.map(lambda x:x+[is_weekend(x[4])])\
                    .map(lambda x:[int(x[0]),int(x[1]),int(x[2])
                                   ,x[4],x[5],x[6]])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
status_rdd_week.cache().take(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[2, 2, 25, datetime.datetime(2013, 8, 29, 12, 0), 12, False], [2, 2, 25, datetime.datetime(2013, 8, 29, 12, 0), 12, False], [2, 2, 25, datetime.datetime(2013, 8, 29, 12, 0), 12, False]]

In [50]:
schema = StructType([StructField("station_id", IntegerType(), True),
                     StructField("bikes_available", IntegerType(), False),
                     StructField("docks_available", IntegerType(), True),
                     StructField("time", DateType(), True),
                     StructField("hour", IntegerType(), False),
                     StructField("is_weekend", BooleanType(), True),
                     ])
df_status = ss.createDataFrame(status_rdd_week, schema)
df_status.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[station_id: int, bikes_available: int, docks_available: int, time: date, hour: int, is_weekend: boolean]

In [51]:
df_status = df_status.withColumnRenamed('time','date')
df_status.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+---------------+----------+----+----------+
|station_id|bikes_available|docks_available|      date|hour|is_weekend|
+----------+---------------+---------------+----------+----+----------+
|         2|              2|             25|2013-08-29|  12|     false|
|         2|              2|             25|2013-08-29|  12|     false|
|         2|              2|             25|2013-08-29|  12|     false|
|         2|              2|             25|2013-08-29|  12|     false|
|         2|              2|             25|2013-08-29|  12|     false|
+----------+---------------+---------------+----------+----+----------+
only showing top 5 rows

In [52]:
status_grop = df_status.groupBy('station_id','date','hour')\
    .agg(avg('docks_available'),avg('bikes_available'),min('is_weekend'))
status_grop.cache().show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+--------------------+--------------------+---------------+
|station_id|      date|hour|avg(docks_available)|avg(bikes_available)|min(is_weekend)|
+----------+----------+----+--------------------+--------------------+---------------+
|         2|2013-08-31|   4|                16.0|                11.0|           true|
|         2|2013-09-27|  19|                14.0|                13.0|          false|
|         2|2013-10-17|  16|  10.983333333333333|  16.016666666666666|          false|
|         2|2013-10-26|   3|                 6.0|                21.0|           true|
|         2|2013-11-03|  17|  10.566666666666666|  16.433333333333334|           true|
+----------+----------+----+--------------------+--------------------+---------------+
only showing top 5 rows

In [58]:
status_grop.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['station_id', 'date', 'hour', 'avg(docks_available)', 'avg(bikes_available)', 'min(is_weekend)']

In [62]:
status_grop = status_grop.withColumnRenamed('avg(docks_available)','docks_available')\
                .withColumnRenamed('avg(bikes_available)','bikes_available')\
                .withColumnRenamed('s_weekend','is_weekend')
status_grop.show(5)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+------------------+------------------+----------+
|station_id|      date|hour|   docks_available|   bikes_available|is_weekend|
+----------+----------+----+------------------+------------------+----------+
|         2|2013-08-31|   4|              16.0|              11.0|      true|
|         2|2013-09-27|  19|              14.0|              13.0|     false|
|         2|2013-10-17|  16|10.983333333333333|16.016666666666666|     false|
|         2|2013-10-26|   3|               6.0|              21.0|      true|
|         2|2013-11-03|  17|10.566666666666666|16.433333333333334|      true|
+----------+----------+----+------------------+------------------+----------+
only showing top 5 rows

In [122]:
status_grop.orderBy('station_id','date','hour').show(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+------------------+------------------+----------+
|station_id|      date|hour|   docks_available|   bikes_available|is_weekend|
+----------+----------+----+------------------+------------------+----------+
|         2|2013-08-29|  12|              25.0|               2.0|     false|
|         2|2013-08-29|  13| 24.30188679245283|  2.69811320754717|     false|
|         2|2013-08-29|  14|              25.0|               2.0|     false|
|         2|2013-08-29|  15|              25.0|               2.0|     false|
|         2|2013-08-29|  16|              25.0|               2.0|     false|
|         2|2013-08-29|  17| 24.09259259259259|2.9074074074074074|     false|
|         2|2013-08-29|  18|              24.0|               3.0|     false|
|         2|2013-08-29|  19|24.454545454545453|2.5454545454545454|     false|
|         2|2013-08-29|  20|25.403846153846153|1.5961538461538463|     false|
|         2|2013-08-29|  21|              26.0|               1.

In [63]:
status_grop.write.mode("overwrite")\
        .parquet("s3://msds694.proj/data/parquet/status_grouped")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [144]:
status_weather = status_grop.join(weather_df_processed,'date','left_outer')
status_weather.cache().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+------------------+------------------+----------+----------+---------------+
|      date|station_id|hour|   docks_available|   bikes_available|is_weekend|temp_float|rain_identifier|
+----------+----------+----+------------------+------------------+----------+----------+---------------+
|2014-02-16|        13|   0| 7.066666666666666| 7.933333333333334|      true|      56.0|           true|
|2014-02-16|        26|  13|               7.0|               8.0|      true|      56.0|           true|
|2014-02-16|        28|   0|              11.0|              12.0|      true|      56.0|           true|
|2014-02-16|        35|   5|               5.0|               6.0|      true|      56.0|           true|
|2014-02-16|        61|   4|             13.25|             13.75|      true|      56.0|           true|
|2014-02-16|        61|  13|11.083333333333334|15.916666666666666|      true|      56.0|           true|
|2014-02-16|        66|  13|10.333333333333334| 8.66666

In [145]:
status_weather.orderBy('station_id','date','hour').show(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+------------------+------------------+----------+----------+---------------+
|      date|station_id|hour|   docks_available|   bikes_available|is_weekend|temp_float|rain_identifier|
+----------+----------+----+------------------+------------------+----------+----------+---------------+
|2013-08-29|         2|  12|              25.0|               2.0|     false|      70.4|          false|
|2013-08-29|         2|  13| 24.30188679245283|  2.69811320754717|     false|      70.4|          false|
|2013-08-29|         2|  14|              25.0|               2.0|     false|      70.4|          false|
|2013-08-29|         2|  15|              25.0|               2.0|     false|      70.4|          false|
|2013-08-29|         2|  16|              25.0|               2.0|     false|      70.4|          false|
|2013-08-29|         2|  17| 24.09259259259259|2.9074074074074074|     false|      70.4|          false|
|2013-08-29|         2|  18|              24.0|        

In [93]:
def type_safe(v):
    try:
        return int(v)
    except ValueError:
        return str(v) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
trip_rdd = sc.textFile(trip_path).filter(lambda x: not x.startswith('id'))\
                                  .map(lambda x:x.split(','))\
                                  .map(lambda x: change_col(x,2,parse))\
                                  .map(lambda x: change_col(x,5,parse))\
                                  
trip_rdd.take(3)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['4576', '63', datetime.datetime(2013, 8, 29, 14, 13), 'South Van Ness at Market', '66', datetime.datetime(2013, 8, 29, 14, 14), 'South Van Ness at Market', '66', '520', 'Subscriber', '94127'], ['4607', '70', datetime.datetime(2013, 8, 29, 14, 42), 'San Jose City Hall', '10', datetime.datetime(2013, 8, 29, 14, 43), 'San Jose City Hall', '10', '661', 'Subscriber', '95138'], ['4130', '71', datetime.datetime(2013, 8, 29, 10, 16), 'Mountain View City Hall', '27', datetime.datetime(2013, 8, 29, 10, 17), 'Mountain View City Hall', '27', '48', 'Subscriber', '97214']]

In [94]:
start = trip_rdd.map(lambda x: [type_safe(x[0]), x[2],x[2].hour, type_safe(x[4])])
end = trip_rdd.map(lambda x: [type_safe(x[0]), x[5], x[5].hour,type_safe(x[7])])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
startschema = StructType([
    StructField("trip_id",IntegerType(),False),
    StructField("start_date", DateType(),False),
    StructField("start_hour", IntegerType(),False),
    StructField("start_station_id",IntegerType(),False),
])

endschema = StructType([
    StructField("trip_id",IntegerType(),False),
    StructField("end_date", DateType(),False),
    StructField("end_hour", IntegerType(),False),
    StructField("end_station_id",IntegerType(),False),
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
start_df0 = ss.createDataFrame(start, startschema)
end_df0 = ss.createDataFrame(end, endschema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [98]:
start_result = start_df0.groupby('start_station_id', 'start_date','start_hour').count()
start_result.cache().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------+----------+-----+
|start_station_id|start_date|start_hour|count|
+----------------+----------+----------+-----+
|              65|2015-08-11|        18|    5|
|              63|2015-08-11|        16|    4|
|              54|2015-08-10|        18|    1|
|               6|2015-08-10|        17|    3|
|              68|2015-08-10|        17|    1|
|              50|2015-08-10|        15|    1|
|              45|2015-08-10|         8|    3|
|              82|2015-08-10|         8|    2|
|              59|2015-08-09|        21|    1|
|              68|2015-08-09|        13|    2|
|              58|2015-08-08|        17|    1|
|              57|2015-08-08|        10|    1|
|              46|2015-08-08|         3|    1|
|              71|2015-08-07|         8|    3|
|              72|2015-08-07|         4|    1|
|              41|2015-08-06|        11|    3|
|              62|2015-08-05|        10|    1|
|              49|2015-08-04|        19|    2|
|            

In [99]:
end_result = end_df0.groupby('end_station_id', 'end_date','end_hour').count()
end_result.cache().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+----------+--------+-----+
|end_station_id|  end_date|end_hour|count|
+--------------+----------+--------+-----+
|            49|2013-08-29|      19|    2|
|             3|2013-08-29|      20|    2|
|            66|2013-08-30|      10|    4|
|            28|2013-09-01|      10|    1|
|            47|2013-09-01|      16|    2|
|            48|2013-09-02|      11|    3|
|            76|2013-09-02|      15|    4|
|            38|2013-09-03|      16|    2|
|            76|2013-09-03|      19|    1|
|            47|2013-09-04|       9|    1|
|            41|2013-09-05|      14|    1|
|            10|2013-09-05|      16|    1|
|            65|2013-09-05|      16|    1|
|            49|2013-09-06|       8|    3|
|            63|2013-09-06|      17|    1|
|             8|2013-09-06|      20|    2|
|            71|2013-09-07|       9|    2|
|            60|2013-09-07|      11|    6|
|            16|2013-09-07|      19|    3|
|            50|2013-09-08|      10|    1|
+----------

In [104]:
end_result = end_result.withColumnRenamed('count','end_count')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
 result = start_result.join(end_result, (start_result['start_station_id'] == end_result['end_station_id']) 
                           & (start_result['start_hour'] == end_result['end_hour'])
                            & (start_result['start_date'] == end_result['end_date']),
                            'inner')
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [106]:
result.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------+----------+-----+--------------+----------+--------+---------+
|start_station_id|start_date|start_hour|count|end_station_id|  end_date|end_hour|end_count|
+----------------+----------+----------+-----+--------------+----------+--------+---------+
|              49|2013-08-29|        19|    1|            49|2013-08-29|      19|        2|
|               3|2013-08-29|        20|    1|             3|2013-08-29|      20|        2|
|              66|2013-08-30|        10|    4|            66|2013-08-30|      10|        4|
+----------------+----------+----------+-----+--------------+----------+--------+---------+
only showing top 3 rows

In [107]:
result_df = result.select('start_station_id','start_date', 'start_hour', 'count', 'end_count')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [111]:

result_df = result_df.withColumnRenamed('start_station_id','station_id').withColumnRenamed('start_date','date').withColumnRenamed('start_hour','hour').withColumnRenamed('count','start_count')
result_df.cache().show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+-----------+---------+
|station_id|      date|hour|start_count|end_count|
+----------+----------+----+-----------+---------+
|        65|2015-08-11|  18|          5|        5|
|        63|2015-08-11|  16|          4|        2|
|        54|2015-08-10|  18|          1|        2|
+----------+----------+----+-----------+---------+
only showing top 3 rows

In [146]:
status_weather.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+-----------------+-----------------+----------+----------+---------------+
|      date|station_id|hour|  docks_available|  bikes_available|is_weekend|temp_float|rain_identifier|
+----------+----------+----+-----------------+-----------------+----------+----------+---------------+
|2014-02-16|        13|   0|7.066666666666666|7.933333333333334|      true|      56.0|           true|
|2014-02-16|        26|  13|              7.0|              8.0|      true|      56.0|           true|
|2014-02-16|        28|   0|             11.0|             12.0|      true|      56.0|           true|
|2014-02-16|        35|   5|              5.0|              6.0|      true|      56.0|           true|
|2014-02-16|        61|   4|            13.25|            13.75|      true|      56.0|           true|
+----------+----------+----+-----------------+-----------------+----------+----------+---------------+
only showing top 5 rows

In [147]:
final_df = status_weather.join(result_df,['station_id','date','hour'],'left_outer')
final_df.cache().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+------------------+------------------+----------+----------+---------------+-----------+---------+
|station_id|      date|hour|   docks_available|   bikes_available|is_weekend|temp_float|rain_identifier|start_count|end_count|
+----------+----------+----+------------------+------------------+----------+----------+---------------+-----------+---------+
|        13|2014-02-16|   0| 7.066666666666666| 7.933333333333334|      true|      56.0|           true|       null|     null|
|        26|2014-02-16|  13|               7.0|               8.0|      true|      56.0|           true|       null|     null|
|        28|2014-02-16|   0|              11.0|              12.0|      true|      56.0|           true|       null|     null|
|        35|2014-02-16|   5|               5.0|               6.0|      true|      56.0|           true|       null|     null|
|        61|2014-02-16|   4|             13.25|             13.75|      true|      56.0|           true|       

In [153]:
final_df.orderBy('start_count',ascending=False).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----+------------------+------------------+----------+----------+---------------+-----------+---------+
|station_id|      date|hour|   docks_available|   bikes_available|is_weekend|temp_float|rain_identifier|start_count|end_count|
+----------+----------+----+------------------+------------------+----------+----------+---------------+-----------+---------+
|        70|2015-01-28|   8| 9.516666666666667| 9.483333333333333|     false|      55.4|          false|         52|       15|
|        70|2015-01-07|   8|               9.7|               9.3|     false|      52.4|          false|         45|       21|
|        70|2015-02-18|   8|              12.6| 6.066666666666666|     false|      58.0|          false|         44|       16|
|        69|2015-06-23|   8| 9.666666666666666|13.333333333333334|     false|      64.8|          false|         44|        9|
|        69|2015-03-16|   8|17.016666666666666| 5.916666666666667|     false|      63.0|          false|       

In [154]:
final_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['station_id', 'date', 'hour', 'docks_available', 'bikes_available', 'is_weekend', 'temp_float', 'rain_identifier', 'start_count', 'end_count']

In [160]:
final_group = final_df.groupBy('station_id', 
                 'hour', 
                 'is_weekend', 
                 'temp_float',
                 'rain_identifier').agg(mean('docks_available')
                      ,mean('start_count')
                      ,mean('end_count')
                      ,mean('bikes_available'))
final_group.cache().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+----------+----------+---------------+--------------------+----------------+--------------+--------------------+
|station_id|hour|is_weekend|temp_float|rain_identifier|avg(docks_available)|avg(start_count)|avg(end_count)|avg(bikes_available)|
+----------+----+----------+----------+---------------+--------------------+----------------+--------------+--------------------+
|        16|  10|     false|      71.4|          false|   8.094444444444443|            null|          null|   6.905555555555556|
|        57|  17|     false|      71.4|          false|  10.855555555555554|             4.5|           3.0|   4.144444444444445|
|         3|   3|     false|      71.4|          false|  6.3999999999999995|            null|          null|                 8.6|
|        66|   2|     false|      71.4|          false|   7.333333333333333|            null|          null|  11.666666666666666|
|        22|  17|     false|      57.0|           true|                 6.0|            nu

In [ ]:
||avg(end_count)|avg(bikes_available)

In [164]:
final_group=final_group.withColumnRenamed('avg(docks_available)','docks_available').withColumnRenamed('avg(start_count)','start_count')\
.withColumnRenamed('avg(end_count)','end_count').withColumnRenamed('avg(bikes_available)','bikes_available')
final_group.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+----------+----------+---------------+------------------+-----------+---------+------------------+
|station_id|hour|is_weekend|temp_float|rain_identifier|   docks_available|start_count|end_count|   bikes_available|
+----------+----+----------+----------+---------------+------------------+-----------+---------+------------------+
|        16|  10|     false|      71.4|          false| 8.094444444444443|       null|     null| 6.905555555555556|
|        57|  17|     false|      71.4|          false|10.855555555555554|        4.5|      3.0| 4.144444444444445|
|         3|   3|     false|      71.4|          false|6.3999999999999995|       null|     null|               8.6|
|        66|   2|     false|      71.4|          false| 7.333333333333333|       null|     null|11.666666666666666|
|        22|  17|     false|      57.0|           true|               6.0|       null|     null|              21.0|
+----------+----+----------+----------+---------------+-----------------

In [40]:
# save to our s3 bucket
#final_group.write.mode("overwrite")\
        #.parquet("s3://msds694.proj/data/parquet/final_group_1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
final_df.write.option("path", "/df").saveAsTable('final_data_frame')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df=ss.sql("select * from parquet. `s3://msds694.proj/data/parquet/final_group`")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
df=ss.read.parquet("s3://msds694.proj/data/parquet/final_group")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# group-project assignment 2 stops here

# Machine learning

In [6]:
df=ss.read.parquet("s3://msds694.proj/data/parquet/final_group")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df.groupBy(df['start_count']).count().orderBy('count',ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------+
|       start_count| count|
+------------------+------+
|              null|409306|
|               1.0| 31727|
|               2.0| 17190|
|               3.0|  7908|
|               1.5|  4216|
|               4.0|  4017|
|               2.5|  2354|
|               5.0|  2244|
|1.6666666666666667|  1343|
|               6.0|  1239|
|1.3333333333333333|  1199|
|               3.5|  1105|
|2.3333333333333335|   978|
|2.6666666666666665|   782|
|               7.0|   721|
|               4.5|   615|
|3.3333333333333335|   492|
|              1.75|   461|
|               8.0|   442|
|              2.25|   399|
+------------------+------+
only showing top 20 rows

In [8]:
dfrp=df.na.fill(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
dfrp.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+----------+----------+---------------+------------------+-----------+---------+------------------+
|station_id|hour|is_weekend|temp_float|rain_identifier|   docks_available|start_count|end_count|   bikes_available|
+----------+----+----------+----------+---------------+------------------+-----------+---------+------------------+
|        28|   0|      true|      56.0|           true|              11.0|        0.0|      0.0|              12.0|
|        74|  13|      true|      56.0|           true|              10.0|        0.0|      0.0|              13.0|
|        21|  17|      true|      56.0|           true|               8.0|        0.0|      0.0|               7.0|
|        48|  20|      true|      56.0|           true|              10.0|        0.0|      0.0|               5.0|
|        32|  23|      true|      56.0|           true|               5.0|        0.0|      0.0|               6.0|
|        77|  18|      true|      56.0|           true|2.366666666666666

In [10]:
dfrp1=dfrp.withColumn('is_weekend',dfrp['is_weekend'].cast('integer'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
dfrp1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+----------+----------+---------------+------------------+-----------+---------+------------------+
|station_id|hour|is_weekend|temp_float|rain_identifier|   docks_available|start_count|end_count|   bikes_available|
+----------+----+----------+----------+---------------+------------------+-----------+---------+------------------+
|        28|   0|         1|      56.0|           true|              11.0|        0.0|      0.0|              12.0|
|        74|  13|         1|      56.0|           true|              10.0|        0.0|      0.0|              13.0|
|        21|  17|         1|      56.0|           true|               8.0|        0.0|      0.0|               7.0|
|        48|  20|         1|      56.0|           true|              10.0|        0.0|      0.0|               5.0|
|        32|  23|         1|      56.0|           true|               5.0|        0.0|      0.0|               6.0|
|        77|  18|         1|      56.0|           true|2.366666666666666

In [12]:
#converting strings to numeric values
from pyspark.ml.feature import StringIndexer

def indexStringColumns(df, cols):
    #variable newdf will be updated several times
    newdf = df
    
    for c in cols:
        #For each given colum, fits StringIndexerModel.
        si = StringIndexer(inputCol=c, outputCol=c+"-num",stringOrderType="alphabetAsc")
        sm = si.fit(newdf)
        #Creates a DataFame by putting the transformed values in the new colum with suffix "-num" 
        #and then drops the original columns.
        #and drop the "-num" suffix. 
        newdf = sm.transform(newdf).drop(c)
        newdf = newdf.withColumnRenamed(c+"-num", c)
    return newdf

dfnumeric = indexStringColumns(dfrp1, ['rain_identifier'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
dfnumeric.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+----------+----------+---------------+-----------+---------+---------------+---------------+
|station_id|hour|is_weekend|temp_float|docks_available|start_count|end_count|bikes_available|rain_identifier|
+----------+----+----------+----------+---------------+-----------+---------+---------------+---------------+
|        28|   0|         1|      56.0|           11.0|        0.0|      0.0|           12.0|            1.0|
|        74|  13|         1|      56.0|           10.0|        0.0|      0.0|           13.0|            1.0|
|        21|  17|         1|      56.0|            8.0|        0.0|      0.0|            7.0|            1.0|
+----------+----+----------+----------+---------------+-----------+---------+---------------+---------------+
only showing top 3 rows

In [14]:
df_model=dfnumeric.select('station_id','hour','is_weekend','temp_float','docks_available','start_count','end_count'\
                   ,'rain_identifier','bikes_available')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_model=df_model.withColumn('bikes_available',dfrp['bikes_available'].cast('integer'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_model.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----+----------+----------+---------------+-----------+---------+---------------+---------------+
|station_id|hour|is_weekend|temp_float|docks_available|start_count|end_count|rain_identifier|bikes_available|
+----------+----+----------+----------+---------------+-----------+---------+---------------+---------------+
|        28|   0|         1|      56.0|           11.0|        0.0|      0.0|            1.0|             12|
|        74|  13|         1|      56.0|           10.0|        0.0|      0.0|            1.0|             13|
|        21|  17|         1|      56.0|            8.0|        0.0|      0.0|            1.0|              7|
+----------+----+----------+----------+---------------+-----------+---------+---------------+---------------+
only showing top 3 rows

In [17]:
# Merging the data with Vector Assembler.
va = VectorAssembler(outputCol="features", inputCols=dfnumeric.columns[0:-1]) #except the last col.
bikepoints = va.transform(df_model).select("features", "bikes_available")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
bikepoints.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------+
|            features|bikes_available|
+--------------------+---------------+
|[28.0,0.0,1.0,56....|             12|
|[74.0,13.0,1.0,56...|             13|
|[21.0,17.0,1.0,56...|              7|
+--------------------+---------------+
only showing top 3 rows

In [19]:
bikepoints=bikepoints.withColumnRenamed('bikes_available','label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Create Training and Test data.
biketsets = bikepoints.randomSplit([0.8, 0.2])
biketrain = biketsets[0].cache()
bikevalid = biketsets[1].cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Decision Tree

In [21]:
# Train the data.
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(maxDepth=20, maxBins= 32, minInstancesPerNode=1, minInfoGain = 0)
dtmodel = dt.fit(biketrain)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
dtpredicts = dtmodel.transform(bikevalid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
metrics = MulticlassClassificationEvaluator()\
                .setLabelCol("label")\
                .setPredictionCol("prediction")
metrics.setMetricName("accuracy") # "f1", "weightedPrecision", "weightedRecall", "accuracy" (defulat)
metrics.evaluate(dtpredicts)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9996179522239202

In [28]:
metrics.setMetricName("weightedRecall") # "f1", "weightedPrecision", "weightedRecall", "accuracy" (defulat)
metrics.evaluate(dtpredicts)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9996179522239201

In [27]:
metrics.setMetricName("f1")
metrics.evaluate(dtpredicts)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9996209305097534

# random Forest

In [98]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(maxDepth=5, numTrees=10,maxBins=2, seed=7)
rfmodel = rf.fit(biketrain)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
validpredicts = rfmodel.transform(bikevalid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
f1=MulticlassClassificationEvaluator().setMetricName("f1").evaluate(validpredicts)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
f1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.23709162148594365

In [116]:
weightedPrecision=MulticlassClassificationEvaluator().setMetricName("weightedPrecision").evaluate(validpredicts)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.20093058576376444

# H2O

In [30]:
sc.install_pypi_package('h2o-pysparkling-2.4')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package already installed for current Spark context!
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1110, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [31]:
sc.install_pypi_package('h2o')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Created wheel for h2o: filename=h2o-3.28.0.1-py2.py3-none-any.whl size=123410554 sha256=131b484e78dd5470e077197a10e6c693dd5d1203d3e85df0ceab2f3fbc4c2750
  Stored in directory: /var/lib/livy/.cache/pip/wheels/ff/82/f4/3fe9c895b93c53b25ae44dc44aa54dad571c711f48bc485fe9
Successfully built h2o

In [33]:
from pysparkling import *
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
import h2o

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
h2o.init()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_232"; OpenJDK Runtime Environment (build 1.8.0_232-b09); OpenJDK 64-Bit Server VM (build 25.232-b09, mixed mode)
  Starting server from /mnt/tmp/1579129711570-0/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpzw9_rdem
  JVM stdout: /tmp/tmpzw9_rdem/h2o_livy_started_from_python.out
  JVM stderr: /tmp/tmpzw9_rdem/h2o_livy_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
--------------------------  ------------------------------------------------------------------
H2O cluster uptime:         01 secs
H2O cluster timezone:       Etc/UCT
H2O data parsing timezone:  UTC
H2O cluster version:        3.28.0.1
H2O cluster version age:    30 days
H2O cluster name:           H2O_from_python_livy_1q2kp0
H2O cluster tot

In [34]:
hc = H2OContext.getOrCreate(ss)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Unsupported argument: (spark.dynamicAllocation.enabled,true)'
Traceback (most recent call last):
  File "/tmp/1579133929596-0/local/lib/python3.6/site-packages/ai/h2o/sparkling/H2OContext.py", line 135, in getOrCreate
    jhc = jvm.org.apache.spark.h2o.JavaH2OContext.getOrCreate(jspark_session, selected_conf._jconf)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 79, in deco
    raise IllegalArgumentException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.IllegalArgumentException: 'Unsupported argument: (spark.dynamicAllocation.enabled,true)'



In [18]:
h2o.import_file("s3://msds694.proj/data/parquet/final_group")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

HTTP 500 Server Error:
Server error water.util.DistributedException:
  Error: DistributedException from /127.0.0.1:54321: 'This H2O node couldn't find the file(s) to parse. Please check files and/or working directories.'
  Request: None

Traceback (most recent call last):
  File "/tmp/1579129711570-0/local/lib/python3.6/site-packages/h2o/h2o.py", line 476, in import_file
    skipped_columns, custom_non_data_line_markers)
  File "/tmp/1579129711570-0/local/lib/python3.6/site-packages/h2o/frame.py", line 430, in _import_parse
    skipped_columns, custom_non_data_line_markers)
  File "/tmp/1579129711570-0/local/lib/python3.6/site-packages/h2o/frame.py", line 444, in _parse
    skipped_columns, custom_non_data_line_markers)
  File "/tmp/1579129711570-0/local/lib/python3.6/site-packages/h2o/h2o.py", line 769, in parse_setup
    j = api("POST /3/ParseSetup", data=kwargs)
  File "/tmp/1579129711570-0/local/lib/python3.6/site-packages/h2o/h2o.py", line 123, in api
    return h2oconn.request(en